# SNODAS Wrapper

This code was written by Jonah Joughin with modifications by A. Arendt

We're aware of a few other repos that overlap with this work:

* David Hill's [matlab and shell scripts](https://github.com/dfosterhill/SNODAS)
* David Shean's [Python snowtools](https://github.com/dshean/snowtools)

In [1]:
%matplotlib inline
import re
import os
import math
import xarray as xr
import numpy as np
import dask
from datetime import datetime, timedelta
import seaborn as sns
import matplotlib.pyplot as plt
from validation import SNODAS, MountainHub, Elevation, utils as ut

## Downloading Data from SNODAS
Data can be fetched from SNODAS using the `snodas_ds(date)` function. It can then be saved using the appropriate function e.g. `save_netcdf(dataset, path)` or `save_tiff(dataset, path)`

In [ ]:
# Fetch data from SNODAS
start_date = datetime(2019,9,14)
for date in (start_date + timedelta(n) for n in range(180)):
    output_path = date.strftime('data/SNODAS/SNODAS_%Y%m%d.nc')
    print(output_path)
    if not os.path.exists(output_path):
        snodas_ds = SNODAS.snodas_ds(date)
        ut.save_netcdf(snodas_ds, output_path)

## Merging files into single NetCDF files
Merging together NetCDF files for different dates can be accomplished by reading those files into xarray, and then writing the resulting dataset out to a new NetCDF file. It is important to set the proper coordinates along the time axis of the dataset before writing, in order to record the date of each individual layer. 

In [ ]:
output_path = 'data/SNODAS/SNODAS_2019_sample.nc'
# Only process if file does not already exist
if not os.path.exists(output_path):
    # Get list of valid files
    pattern = re.compile("^SNODAS_\d{8}.nc$")
    # There's something wrong with SNODAS-20170913 - only use files after this for now
    files = sorted([os.path.join("data/SNODAS",f) for f in os.listdir("data/SNODAS") if pattern.match(f)])
    # Extract dates from files
    dates = [ut.date_from_file(f) for f in files]

    ds = xr.open_mfdataset(files, concat_dim='time', combine='nested')
    ds.coords['time'] = dates
    
    ds.to_netcdf(output_path)

### MISSING MASKED FILES

These notes are from David Hill's [shell script](https://github.com/dfosterhill/SNODAS/blob/master/get_proc_snodas.sh)

### The following dates are missing ALL data (YYYY-MM-DD):

* 2004-02-25
* 2004-08-31
* 2004-09-27
* 2005-06-25
* 2005-08-01
* 2005-08-02
* 2006-08-26
* 2006-08-27
* 2006-09-08
* 2006-09-30
* 2006-10-01
* 2007-02-14
* 2007-03-26
* 2008-03-13
* 2008-06-13
* 2008-06-18
* 2009-08-20
* 2012-12-20

### The following dates are missing individual files:

2003-10-30 is missing one file:

* us_ssmv11034tS__T0001TTNATS2003103005HP001

## Loading Merged Data in XArray
The merged dataset can then be reimported to xarray with the following function. The merging process is important, as it allows xarray to search the dataset much faster, opening up the possibility of API queries.

In [ ]:
ds = xr.open_dataset('data/SNODAS/SNODAS_2017_2018.nc')

## Selecting snow depth across region
Data can be selected in a particular region by using the `sel` and `isel` functions in conjunction with `slice`, as shown below.

In [ ]:
ds.Band1.sel(time = '2017-12-31', lat = slice(40, 54), lon = slice(-126, -110)).plot()

## Selecting time series at location
A time series can be constructed as follows. It is important that the method parameter is set to `'nearest'` in order to recieve results.

In [ ]:
lat = 44.0018751
lon = -122.004

region = {
    'ymax' : 44.25,
    'ymin' : 43.75,
    'xmax': -121.65,
    'xmin': -122.5,
}

obs = MountainHub.snow_data(limit=1000, start=datetime(2017,8,1), end=datetime(2018,3,28), box=region)
obs = Elevation.merge_el_data(obs)

plot = obs.plot(x='date', y='snow_depth', style='o')
snodas_series = ds.Band1.sel(lat=lat, lon=-122, method='nearest') / 10
snodas_series.plot(ax=plot)


## Selecting point data at time and location
Data can be easily selected for a particular location and date. In the example below, snow depth is retrieved for January 3, 2018 near 44° N, 121.65° W.

In [ ]:
ds.sel(time = '2018-1-3', lon = -121.65, lat = 44, method='nearest').Band1.item()

## Plotting Data Against SNODAS

In [ ]:
# Restrict points to the continental US
region = {
    'ymax' : 50,
    'ymin' : 25,
    'xmax': -65,
    'xmin': -125,
}

obs = MountainHub.snow_data(limit=1000, start=datetime(2017,9,14), end=datetime(2018,3,28), box=region)
def snodas_depth(ts, lon, lat):
    height = ds.Band1.sel(time = ts.strftime('%Y-%m-%d'), lon=lon, lat=lat, method='nearest').item()
    if not np.isnan(height):
        height /= 10
    return height

obs['snodas_depth'] = obs.apply(lambda x: snodas_depth(x['date'], x['long'], x['lat']), axis=1)

sns.set(color_codes=True)
sns.lmplot(x='snodas_depth',y='snow_depth',data=obs, fit_reg=True, lowess=True) 